In [7]:
import pygame as pg
import sys
import random
import numpy as np

In [8]:
def obstacle_render(obs_list,obst_list):
    if obs_list:
        for i in range (len(obs_list)):
            screen.blit(pipe,pipe.get_rect(topleft=(obs_list[i][0],obs_list[i][1])))
            screen.blit(pipet,pipet.get_rect(topleft=obst_list[i]))
            obst_list[i][0]=obst_list[i][0]-4
            obs_list[i][0]=obst_list[i][0]-4
        if obs_list[0][0]<-100:
            del obs_list[0]
            del obst_list[0]
        return obs_list,obst_list
    else:
        return[],[]

In [9]:
# x = horizontal distance between birdcenter and the pipe left range={0,4,....,604}
# y = vertical distance of birdcenter and bottom pipe top range=[-301,475]
# h = difference between the top and bottom pipes range={10,40}
# z = velocity of the bird range=[-47,21]
# last one denotes action
Q_value=np.zeros((152,777,69,2,2),dtype=np.float32)

In [10]:
eps=0.001
td_error=0.01
Y=0.5

In [11]:
def get_next_state(cur,obs_list):
    nexts=[0,0,0,0]
    if cur[4]==0:
        nexts[1]=int(pipefinal[1]-(s+(t+0.5)*(t+0.5)-(t+0.5)*a)+301)
        nexts[2]=a-(t+0.5)*2+47
    else:
        nexts[1]=int(pipefinal[1]-(s+t*t-t*a + 0.5*0.5 -0.5*21)+301)
        nexts[2]=21+47
    if cur[0]>0:
        
        nexts[0]=cur[0]-1
        nexts[3]=cur[3]
    else:
        nexts[0]=(obs_list[1][0]-100)/4
        if obs_list[1][2]==40:
            nexts[3]=1
        else:
            nexts[3]=0
    for i in range(4):
        nexts[i]=int(nexts[i])
    return nexts

In [ ]:
pg.init()
screen=pg.display.set_mode((1200,576))
pg.display.set_caption("Hello3")
clock=pg.time.Clock()
font = pg.font.Font('freesansbold.ttf', 22)
sky=pg.image.load('game_dev/finalbg.jpg').convert()
pipe=pg.image.load('game_dev/pipe.png').convert_alpha()
pipe=pg.transform.scale(pipe,(100,500))
pipet=pg.transform.rotate(pipe,180)
pipet=pg.transform.flip(pipet,True,False)
pipet_rect=pipet.get_rect(topleft=(1200,-50))
pipe_rect=pipe.get_rect(topleft=(1200,300))
pipe_rect_copy=pipe.get_rect(topleft=(550,300))
pipet_rect_copy=pipet.get_rect(topleft=(550,-50))
bird=pg.image.load('game_dev/bird.png').convert_alpha()
bird=pg.transform.scale(bird,(70,50))
bird_rect=bird.get_rect(center=(100,100))
sbt=font.render("START",True,"red")
sbt_rect=sbt.get_rect(center=(600,288))
screen.blit(sbt,sbt_rect)

#defining timer for the obstacles to occur
for i in range(10000):
    j=1
    timestamp=0
    bird_rect=bird.get_rect(center=(100,400))
    timer=pg.USEREVENT+1
    pg.time.set_timer(timer,2000)

    #bird=pg.transform.rotate(bird,-30)
    obs_list=[[400,400,10],[900,300,40]]
    obst_list=[[400,400-700+10],[900,300-700+40]]
    t=0
    a=0
    score="0"
    s=bird_rect.centery
    x1=random.randint(-250,0)
    x2=random.randint(-300,-50)
    text = font.render(score, True, (77,208,225))
    text_rect = text.get_rect(center=(600,80))
    l=[]
    el=pg.draw.circle(screen,"blue",bird_rect.center,20)
    done=False
    while not done:
        #draw everything and update
        for event in pg.event.get():
            if event.type==pg.QUIT:
                print(l)
                pg.quit()
                sys.exit()
            #if event.type==pg.KEYDOWN:
                #s=s+t*t-t*a
                #a=21
                #t=0.5
            if event.type==timer:
                x=4*random.randint(300,330)
                y=400+random.randint(-150,100)
                pipedist=10
                if random.randint(0,50)>25:
                    pipedist=40
                obs_list.append([x,y,pipedist])
                obst_list.append([x,y-700+pipedist])
        # defining thr rectangle postions
        el=pg.draw.circle(screen,"blue",bird_rect.center,20)

        #displaying the final position on the surface
        screen.blit(sky,(-100,0))
        bird_rect.centery=s+t*t-t*a
        v=a-t*2
        screen.blit(bird,bird_rect)
        screen.blit(text,text_rect)

        obs_list,obst_list=obstacle_render(obs_list,obst_list)

        if j==1:
            if obs_list[0][0]<100:
                pipefinal=obs_list[1]
            else:
                pipefinal=obs_list[0]
            if pipefinal[2]==40:
                state_h=1
            else:
                state_h=0

            state_y=pipefinal[1]-bird_rect.centery+301
            state_z=v+47
            if obs_list[0][0]<100:
                pipefinal=obs_list[1]
            else:
                pipefinal=obs_list[0]

            state_x=(-bird_rect.centerx+pipefinal[0])/4
            cur_state=[int(state_x),int(state_y),int(state_z),int(state_h)]
            j=0
        if obs_list[0][0]<100:
            pipefinal=obs_list[1]
        else:
            pipefinal=obs_list[0]
                
        if random.uniform(0,1)<eps:
            action=random.randint(0,1)
        else:
            action=np.argmax(Q_value[tuple(cur_state)])
        cur_state.append(action)
        
        next_state=get_next_state(cur_state,obs_list)
        
        # in one time stamp x reduces by one and y by an amout specified by an equation of motion
        # so in order to find the subsequent states i would need to do a lookahead in the s using a hypothetical 
        # jump or no jump and then check the max of q values of both states and decide greedly....

        
        
        if cur_state[4]==0:
            ys=s+(t+0.5)*(t+0.5)-(t+0.5)*a
        else:
            ys=s+t*t-t*a + 0.5*0.5 -0.5*21
        el1=pg.draw.circle(screen,"blue",(104,ys),20)
        if  el1.colliderect(pipe.get_rect(topleft=(obs_list[0][0],obs_list[0][1]))) or el1.colliderect(pipet.get_rect(topleft=obst_list[0])) or el1.bottom>=571 or el1.top<=5:
            reward=-1000
            done=True
            print(cur_state[0])
            print(t)
            Q_value[tuple(cur_state)]+=td_error*(reward)
        else:
            if random.uniform(0,1)<eps:
                action_next=random.randint(0,1)
            else:
                action_next=np.argmax(Q_value[tuple(cur_state)])
            next_state.append(action_next)
            reward=1
            Q_value[tuple(cur_state)]+=td_error*(reward + Y*(Q_value[tuple(next_state)]-Q_value[tuple(cur_state)]))

        if action==1:
            s=s+t*t-t*a
            t=0
            a=21

        if obs_list and el.right==obs_list[0][0]+16:
            score=str(eval(score)+1)
            text = font.render(score, True,(77,208,225))
        #if (obs_list and obst_list and (el.colliderect(pipe.get_rect(topleft=obs_list[0])) or el.colliderect(pipet.get_rect(topleft=obst_list[0])))) or bird_rect.bottom>=576 or bird_rect.top<=0:
            #pg.quit()
            #sys.exit()

        t=t+0.5
        pg.display.update()
        clock.tick(60) 
        timestamp+=1
        cur_state=[next_state[0],next_state[1],next_state[2],next_state[3]]


49
12.0
7
21.0
7
21.5
5
1.0
5
1.5
5
1.5
5
0.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
14
26.0
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
0.5
5
1.5
5
1.0
5
1.0
5
0.5
5
1.0
5
1.0
5
1.0
5
1.0
5
1.0
5
1.0
5
0.5
5
1.0
5
1.0
5
0.5
5
1.0
5
1.0
5
20.0
5
1.0
5
0.5
5
1.0
5
1.0
5
0.5
5
1.0
5
1.0
5
0.5
5
1.0
5
0.5
5
1.0
5
1.0
5
0.5
5
1.0
5
0.5
5
1.0
5
1.0
5
0.5
5
0.5
5
1.5
5
1.0
